In [54]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 

In [55]:
tweets_df = pd.read_csv('../data/tweets_with_sentiment_and_topic.csv')

tweets_df.head()

,Unnamed: 0,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score,top_topic,Cryptocurrency and Fintech,Technology,Healthcare and Pharmaceuticals,Defense and Aerospace,Financials,Airlines and Travel,Energy,Agriculture,Automotive
0,0,1603456812723933194,12.0,jack,"RT @ODELL: @nobsbitcoin love to see it, if yall want to learn more about nostr we had a great conversation about it on @citadeldispatch bac…",en,2022-12-15 18:27:49.597000+00:00,Neutral,0.977740,Cryptocurrency and Fintech,0.757990,0.376340,0.045903,0.032895,0.019666,0.019118,0.013074,0.010588,0.007226
1,1,1600412216687149056,12.0,jack,RT @TheGuySwann: It would be so amazing to see Africa lead the world into a #Bitcoin future while the arrogance of western governments leav…,en,2022-12-07 08:49:41.336000+00:00,Positive,0.999987,Cryptocurrency and Fintech,0.868653,0.853358,0.025603,0.044147,0.210190,0.011476,0.110531,0.037469,0.007175
2,2,1595502702854692866,12.0,jack,RT @kendricklamar: big steppers tour (directors cut) @PrimeVideo https://t.co/LkJuKSDIHG https://t.co/PHvyr5U1bN,en,2022-11-23 19:41:02.024000+00:00,Neutral,0.999613,Technology,0.012002,0.252524,0.057174,0.072260,0.034317,0.007996,0.006217,0.003851,0.017827
3,3,1589450807216599040,12.0,jack,@litcapital @elonmusk Sure,en,2022-11-07 02:52:57.665000+00:00,Neutral,0.985752,Technology,0.284053,0.768173,0.263169,0.119607,0.364800,0.058842,0.239539,0.033245,0.125718
4,4,1582442966521503745,12.0,jack,RT @bluesky: Bluesky is building a social protocol. We released “ADX” (the X stood for Experiment) in May. Now that the design is starting…,en,2022-10-18 18:46:18.244000+00:00,Neutral,0.999923,Technology,0.000134,0.058334,0.002962,0.001737,0.000540,0.004032,0.008653,0.003117,0.000706


In [56]:
print(tweets_df[tweets_df.handle == 'elonmusk'].head())

      Unnamed: 0                   id     user_id    handle  \
2779        2779  1635494082545070081  44196397.0  elonmusk   
2780        2780  1634686692484345857  44196397.0  elonmusk   
2781        2781  1634659471803785223  44196397.0  elonmusk   
2782        2782  1634658739335704577  44196397.0  elonmusk   
2783        2783  1633147320056557569  44196397.0  elonmusk   

                                                                                        clean_text  \
2779  RT @CommunityNotes: Sign up to become a Community Notes contributor: https://t.co/FSsqNzonnz   
2780                                             And the Catfish🥇goes to … https://t.co/u0STEHfxXv   
2781                                                                                     … market?   
2782                                                                       https://t.co/aicsmUfVCR   
2783                                                              @BillyM2k He’s the worst, sorry.   

     lan

In [57]:
prices_df = pd.read_parquet('../data/taq/SYM_ROOT=TSLA')

prices_df.head()

,timestamp,avg_price,year
0,2010-06-29 11:25:00,18.835687,2010
1,2010-06-29 11:26:00,18.550146,2010
2,2010-06-29 11:27:00,18.165070,2010
3,2010-06-29 11:28:00,18.158299,2010
4,2010-06-29 11:29:00,18.229746,2010


In [58]:
print(prices_df.head())

            timestamp  avg_price  year
0 2010-06-29 11:25:00  18.835687  2010
1 2010-06-29 11:26:00  18.550146  2010
2 2010-06-29 11:27:00  18.165070  2010
3 2010-06-29 11:28:00  18.158299  2010
4 2010-06-29 11:29:00  18.229746  2010


In [59]:
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import statsmodels.api as sm
import statsmodels.formula.api as smf


######################################################################
# 0.  CONFIG – easy to change later
######################################################################
TWEET_HANDLE   = 'elonmusk'
STOCK_TICKER   = 'TSLA'
MARKET_TZ      = 'US/Eastern'     # TAQ timestamps are usually New‑York time
TWEET_TZ       = 'UTC'            # your tweets_df timestamps are UTC
TRADING_START  = '09:30'
TRADING_END    = '16:00'

######################################################################
# 1.  LOAD DATA (replace by your own reads)
######################################################################
# tweets_df = pd.read_parquet('tweets.parquet')      # or csv, etc.
# prices_df = pd.read_parquet('tsla_intraday.parquet')

######################################################################
# 2.  CLEAN + ALIGN TIME
######################################################################
tweets_df['timestamp'] = pd.to_datetime(tweets_df['timestamp'], utc=True, format='mixed').dt.tz_convert(MARKET_TZ)
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'], format='%Y-%m-%d %H:%M:%S').dt.tz_localize(MARKET_TZ, ambiguous=True, nonexistent=pd.Timedelta('1h'))

# keep only regular‑session prices (helps a lot)
prices_df = (prices_df
             .set_index('timestamp')
             .between_time(TRADING_START, TRADING_END)
             .tz_localize(None))                          # index without tz is okay from now on

# We’ll resample prices to 1‑min if it isn’t already
prices_1m = prices_df['avg_price'].resample('1min').last().dropna()

######################################################################
# 3.  IDENTIFY EVENT TWEETS
######################################################################
def is_about_stock(row) -> bool:
    text = row['clean_text'].lower()
    if row['handle'] != TWEET_HANDLE:
        return False
    # --- Option A: keyword rule -------------
    keywords = ['tesla', 'tsla', 'gigafactory', 'model s', 'model 3', 'model x', 'model y']
    return any(k in text for k in keywords)
    # --- Option B: topic score -------------
    # return row['Automotive'] >= 0.30

event_tweets = tweets_df.loc[tweets_df.apply(is_about_stock, axis=1)].copy()

# next trading minute **at or after** tweet → align to price index
def align_to_market(ts: pd.Timestamp) -> pd.Timestamp:
    ts_local_naive = ts.tz_localize(None)
    if ts_local_naive in prices_1m.index:
        return ts_local_naive
    return prices_1m.index[prices_1m.index.get_indexer([ts_local_naive], method='bfill')[0]]

event_index = event_tweets['timestamp'].apply(align_to_market)

######################################################################
# 4.  RETURN SERIES + IMPULSE VECTOR
######################################################################
returns = np.log(prices_1m).diff().dropna()       # rₜ
impulse = pd.Series(0.0, index=returns.index)
impulse.loc[event_index] = 1.0                   # weight 1 – change here if you want sentiment

# (Optional) lag the impulse one period to avoid contemporaneous bias
impulse = impulse.shift(1).fillna(0)

######################################################################
# 5.  ARIMA FOR MEAN (normally just AR(1) or none)
######################################################################
arima_res = ARIMA(returns, order=(1,0,0), trend='n').fit()
resid = arima_res.resid

######################################################################
# 6.  GARCH(1,1)-X  (impulse enters the *variance*)
######################################################################
# garch_x = arch_model(resid,
#                      vol='Garch',
#                      p=1, q=1,
#                      mean='Zero',               # mean already handled
#                      x=None,                    # exog for mean – not needed
#                      o=None, power=2.0,
#                      # variance exog is supplied via ‘power’ interface:
#                      # arch>=6.3 allows variance exog via kwargs
#                      variance_kwargs={'x': impulse.values.reshape(-1,1)})

# garch_res = garch_x.fit(disp='off')
# print(garch_res.summary())
# 1. baseline AR(1)+GARCH(1,1)  ⟶  fit with NO exogenous var in variance
base = arch_model(returns, mean='AR', lags=1, vol='Garch', p=1, q=1).fit(disp='off')

# 2. squared STANDARDISED residuals  ε̂² / σ̂²  ≈ 1 when model is perfect
std_resid2 = base.std_resid ** 2

# 3. regress those squares on your tweet‑dummy  (H₀: γ = 0 → no extra volatility)
X = sm.add_constant(impulse.loc[std_resid2.index])   # align indices!
# ols = sm.OLS(std_resid2, X).fit()
df_reg = pd.DataFrame({
    'std_resid2': std_resid2,
    'impulse'   : impulse.loc[std_resid2.index]
}).dropna()

ols = smf.ols('std_resid2 ~ impulse', data=df_reg).fit()

gamma, pval = ols.params['impulse'], ols.pvalues['impulse']
print(f'γ̂ = {gamma:.4g},  p‑value = {pval:.4g}')

######################################################################
# 7.  INTERPRET – p‑value of γ
######################################################################
# gamma = garch_res.params.get('x[1]')           # arch names it automatically
# pval  = garch_res.pvalues.get('x[1]')
print(f'\nγ (impact of tweet on variance) = {gamma:.4g},  p‑value = {pval:.4g}')
if pval < 0.05:
    print('==> Reject H₀: Tweet is associated with a significant volatility change')
else:
    print('==> No statistically significant volatility impact detected')


/Users/clawall/estudo/siads-699/.env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/clawall/estudo/siads-699/.env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/clawall/estudo/siads-699/.env/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/clawall/estudo/siads-699/.env/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check m

γ̂ = -0.1701,  p‑value = 0.9988

γ (impact of tweet on variance) = -0.1701,  p‑value = 0.9988
==> No statistically significant volatility impact detected


/Users/clawall/estudo/siads-699/.env/lib/python3.9/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
